In [20]:
import pandas as pd

In [34]:
KRX_REITs = pd.read_excel('KRX_리츠TOP10지수.xlsx')

KRX_REITs = KRX_REITs.sort_index(ascending=False)
KRX_REITs.reset_index(drop = True, inplace = True)
KRX_REITs.set_index('일자',drop=True, inplace = True)

KRX_REITs_df = KRX_REITs[['종가']]

In [35]:
KRX건설지수 = pd.read_excel('KRX건설지수.xlsx')

KRX건설지수 = KRX건설지수.sort_index(ascending=False)
KRX건설지수.reset_index(drop = True, inplace = True)
KRX건설지수.set_index('일자',drop=True, inplace = True)

KRX건설지수_df = KRX건설지수[['종가']]

In [36]:
BOK뉴스심리지수 = pd.read_excel('BOK뉴스심리지수.xlsx')

BOK뉴스심리지수 = BOK뉴스심리지수.sort_index(ascending=False)
BOK뉴스심리지수.reset_index(drop = True, inplace = True)
BOK뉴스심리지수.set_index('일자',drop=True, inplace = True)

In [37]:
장단기금리차 = pd.read_excel('T10Y2Y.xls')
장단기금리차['observation_date'] = [str(장단기금리차['observation_date'][i])[0:10] for i in range(len(장단기금리차))]
장단기금리차['observation_date'] = 장단기금리차['observation_date'].str.replace('-','/')
장단기금리차.dropna(inplace = True)
장단기금리차.reset_index(drop = True, inplace = True)
장단기금리차.set_index('observation_date',drop=True, inplace = True)

In [39]:
KRX_REITs_df.reset_index(inplace = True)
KRX건설지수_df.reset_index(inplace = True)
장단기금리차.reset_index(inplace = True)
BOK뉴스심리지수.reset_index(inplace = True)

In [40]:
KRX_REITs_df.columns = ['Date', 'KRX리츠TOP지수']
KRX건설지수_df.columns = ['Date', 'KRX건설지수']
장단기금리차.columns = ['Date', 'BOK뉴스심리지수']
BOK뉴스심리지수.columns = ['Date', 'T10Y2Y']

In [41]:
df_final = pd.DataFrame()
df_final = pd.merge(KRX_REITs_df, KRX건설지수_df, how='inner',on=['Date'])
df_final = pd.merge(df_final, BOK뉴스심리지수, how='inner',on=['Date'])
df_final = pd.merge(df_final, 장단기금리차, how='inner',on=['Date'])

In [42]:
df_final

,Date,KRX리츠TOP지수,KRX건설지수,T10Y2Y,BOK뉴스심리지수
0,2022/05/23,1187.49,627.13,108.47,0.21
1,2022/05/24,1194.48,622.74,107.77,0.26
2,2022/05/25,1205.65,632.37,108.64,0.27
3,2022/05/26,1215.53,634.68,109.47,0.29
4,2022/05/27,1220.22,632.63,111.67,0.27
...,...,...,...,...,...
233,2023/05/16,859.21,680.88,94.64,-0.52
234,2023/05/17,855.59,680.59,95.60,-0.55
235,2023/05/18,857.98,686.32,96.50,-0.59
236,2023/05/19,862.43,694.64,99.54,-0.58


In [64]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen


In [65]:
def joh_output(res):
    output = pd.DataFrame([res.lr2,res.lr1],
                          index=['max_eig_stat',"trace_stat"])
    print(output.T,'\n')
    print("Critical values(90%, 95%, 99%) of max_eig_stat\n",res.cvm,'\n')
    print("Critical values(90%, 95%, 99%) of trace_stat\n",res.cvt,'\n')


# model with constant/trend (deterministic) term with lags set to 1
joh_model = coint_johansen(df_final[['KRX리츠TOP지수','KRX건설지수','BOK뉴스심리지수','T10Y2Y']],0,1) # k_ar_diff +1 = K
joh_output(joh_model)

   max_eig_stat  trace_stat
0     37.235065   65.131099
1     14.862405   27.896034
2     10.597136   13.033629
3      2.436493    2.436493 

Critical values(90%, 95%, 99%) of max_eig_stat
 [[25.1236 27.5858 32.7172]
 [18.8928 21.1314 25.865 ]
 [12.2971 14.2639 18.52  ]
 [ 2.7055  3.8415  6.6349]] 

Critical values(90%, 95%, 99%) of trace_stat
 [[44.4929 47.8545 54.6815]
 [27.0669 29.7961 35.4628]
 [13.4294 15.4943 19.9349]
 [ 2.7055  3.8415  6.6349]] 

